In [ ]:
!pip install simpletransformers

In [2]:
import pandas as pd

In [10]:
df = pd.read_csv('train.csv')

Simple transformers requires that a column labels which contains multi-hot encoded lists of labels as well as column text which contains the all the text.

In [11]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
df['text'] = df['comment_text'].apply(lambda x: x.replace('\n', ' '))
df = df[:1000]
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",""" More I can't make any real suggestions on im..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","You, sir, are my hero. Any chance you remember..."


In [12]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)


In [13]:
from simpletransformers.classification import MultiLabelClassificationModel

# Depending on the transformer model to be used just comment the rest

# BERT
#model = MultiLabelClassificationModel('bert', 'bert-base-uncased', num_labels=6, args={'train_batch_size':2, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512, 'fp16' :False, 'save_model_every_epoch' : True})

# XLNet
#model = MultiLabelClassificationModel('xlnet', 'xlnet-base-cased', num_labels=6, args={'train_batch_size':2, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512, 'fp16' :False, 'save_model_every_epoch' : True})

# Distilbert
#model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=6, args={'train_batch_size':2, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512, 'fp16' :False, 'save_model_every_epoch' : False})

# RoBerta
model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'train_batch_size':2, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512, 'fp16' :False, 'save_model_every_epoch' : True})


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [15]:
model.train_model(train_df)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 3:   0%|          | 0/400 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/400 [00:00<?, ?it/s]

(1200, 0.11396143828460481)

In [18]:
test = pd.read_csv('test.csv')
test.head(3)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."


In [19]:

test_df = pd.read_csv('test.csv')

to_predict = test_df.comment_text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

sub_df = pd.DataFrame(outputs, columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])


sub_df['id'] = test_df['id']
sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]

sub_df.to_csv('submission.csv', index=False)


  0%|          | 0/19146 [00:00<?, ?it/s]

In [12]:

result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
print(result)

{'LRAP': 0.9949166666666666, 'eval_loss': 0.07160389955155551}


In [36]:
print(wrong_predictions)

['NA']
